# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



## **Query 1**

**Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

```
Table Name: artist_song_library
column 1: sessionId
column 2: itemInSession
column 3: artist
Column 4: song
Column 5: length
PRIMARY KEY(sessionId, itemInSession)
```

```CQL Query: SELECT artist, song, length FROM artist_song_library WHERE sessionId = 338 AND itemInSession = 4```

Since the query is looking for both sessionId and itemInSession, the Primary Key will be sessionId and itemInSession. In addition, both sessionId and itemInSession are needed to make the row key unique.

**Note**: The sequence of the columns in the CREATE and INSERT statements should follow the order of the COMPOSITE PRIMARY KEY and CLUSTERING columns. The data should be inserted and retrieved in the same order as how the COMPOSITE PRIMARY KEY is set up. This is important because Apache Cassandra is a partition row store, which means the partition key determines which any particular row is stored on which node. In case of composite partition key, partitions are distributed across the nodes of the cluster and how they are chunked for write purposes. Any clustering column(s) would determine the order in which the data is sorted within the partition.

In [8]:
create_artist_song_table_query = "CREATE TABLE IF NOT EXISTS artist_song_library"
create_artist_song_table_query += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_artist_song_table_query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_library (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        try:
            session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
select_artist_song_query =\
"SELECT artist, song, length \
FROM artist_song_library \
WHERE sessionId = %s AND itemInSession = %s"

try:
    rows = session.execute(select_artist_song_query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## **Query 2**

**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182¶**

```
Table Name: artist_song_user_library
column 1: userId
column 2: sessionId
Column 3: itemInSession
column 4: artist
Column 5: user_first_name
Column 6: user_last_name
PRIMARY KEY((userId, sessionId), itemInSession))
```

```CQL Query: SELECT artist, song, user_first_name, user_last_name FROM artist_song_user_library WHERE userId = 10 AND sessionId = 182```

Since the query is looking for both userId and sessionId, the Primary Key will be comprised of the Partition Key (userId and sessionId). The Clustering Column itemInSession will be used to sort the data in ascending order. All 3 columns (userId, sessionId, itemInSession) are needed to make the row key unique.

In [11]:
create_artist_song_user_table_query = "CREATE TABLE IF NOT EXISTS artist_song_user_library"

create_artist_song_user_table_query += "(userId int, sessionId int, itemInSession int, artist text, song text, user_first_name text, user_last_name text, \
PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(create_artist_song_user_table_query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user_library (userId, sessionId, itemInSession, artist, song, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        try:
            session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, user_first_name, user_last_name))
        except Exception as e:
            print(e)

In [13]:
select_artist_song_user_query =\
"SELECT artist, song, user_first_name, user_last_name \
FROM artist_song_user_library \
WHERE userId = %s AND sessionId = %s"

try:
    rows = session.execute(select_artist_song_user_query, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## **Query 3**

**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

```
Table Name: user_song_library
column 1: song
column 2: userId
Column 3: user_first_name
Column 4: user_last_name
PRIMARY KEY(song, userId)
```

```CQL Query: SELECT user_first_name, user_last_name FROM user_song_library WHERE song = 'All Hands Against His Own'```

Since the query is looking for the song 'All Hands Against His Own', the Primary Key can start with the song column. To make the Primary Key unique, we can add the column userId as well.

In [14]:
create_user_song_table_query = "CREATE TABLE IF NOT EXISTS user_song_library"
create_user_song_table_query += "(song text, userId int, user_first_name text, user_last_name text,\
PRIMARY KEY (song, userId))"
try:
    session.execute(create_user_song_table_query)
except Exception as e:
    print(e)           

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_library (song, userId, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        try:
            session.execute(query, (song, int(userId), user_first_name, user_last_name))
        except Exception as e:
            print(e)

In [16]:
select_user_song_query =\
"SELECT user_first_name, user_last_name \
FROM user_song_library \
WHERE song = %s"

try:
    rows = session.execute(select_user_song_query, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE artist_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_song_user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()